In [1]:
from klrw.standard_ebranes import minuscule_weights

In [2]:
cartan_type = CartanType(["B", 2])
Phi = RootSystem(cartan_type)
lambd = minuscule_weights(Phi)[0]

In [3]:
from klrw.standard_ebranes import standard_ebranes

e_brane = standard_ebranes(Phi, lambd)

Finding weight poset
Defining KLRW algebra
Framed quiver:
0   2   
W1  W2  
□   □
|   |
◯=>=◯
V1  V2  
1   2   
Finding stable envlopes
Correcting degree d_{V2,c} + 2*d_{V1--V2} + 2*d_{0}
Start: Finding possible corrections for C_3 -> C_2
Start: Finding possible corrections for C_1 -> C_0
Start: Finding possible corrections for C_2 -> C_1
End: Finding possible corrections for C_3 -> C_2
End: Finding possible corrections for C_1 -> C_0
End: Finding possible corrections for C_2 -> C_1
Making the left hand side for the corrections C_1 -> C_-1
Making the left hand side for the corrections C_2 -> C_0
Making the left hand side for the corrections C_3 -> C_1
Making the right hand side for the corrections C_1 -> C_-1
Making the right hand side for the corrections C_2 -> C_0
Making the right hand side for the corrections C_3 -> C_1
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-15
Set parameter Method to value 0
Gurobi Optimizer version 11.0.1 build v11.

In [4]:
from klrw.framed_dynkin import FramedDynkinDiagram_with_dimensions
KLRW = e_brane.KLRW_algebra
kwrds = KLRW._reduction[2]["quiver_data"]
DD = kwrds
DD_more_strands = FramedDynkinDiagram_with_dimensions = DD.mutable_copy()

for vertex in DD.non_framing_nodes():
    DD_more_strands[vertex] *= 2

DD_more_strands

0   2   
W1  W2  
□   □
|   |
◯=>=◯
V1  V2  
2   4   

In [5]:
from klrw.klrw_algebra import KLRWAlgebra
R = QQ

cls, args, kwrds = e_brane.KLRW_algebra._reduction
R = QQ
kwrds["base_R"] = R
kwrds["quiver_data"] = DD_more_strands
KLRW_new = cls(*args, **kwrds)

In [6]:
min_hom_deg = min(e_brane.projectives.keys())
lowest_projective = e_brane.projectives[min_hom_deg][0]
lowest_state = lowest_projective.state

framing1, *new_sequence, framing2 = lowest_state
new_sequence = tuple(new_sequence)
new_sequence

(V2, V1, V2)

In [7]:
from collections import defaultdict
from klrw.framed_dynkin import DotVariableIndex
from klrw.perfect_complex import KLRWProjectiveModule, KLRWPerfectComplex

projectives_left = {
    i: [
        KLRWProjectiveModule(
            KLRW_new.state(new_sequence + pr.state.as_tuple()),
            pr.equivariant_degree,
            grading_group=KLRW_new.grading_group,
        )
        for pr in projs
    ]
    for i, projs in e_brane.projectives.items()
}

new_strands_count = defaultdict(int)
for x in new_sequence:
    new_strands_count[x] += 1

variables_images = [None] * KLRW.base().ngens()
for index, var in KLRW.base().variables.items():
    if var.position is not None:
        if isinstance(index, DotVariableIndex):
            vertex = index.vertex
            new_index = DotVariableIndex(
                vertex, index.number + new_strands_count[vertex]
            )
        else:
            new_index = index
        variables_images[var.position] = KLRW_new.base().variables[new_index].monomial

move_dots_to_right = KLRW.base().hom(variables_images, codomain=KLRW_new.base())
print(move_dots_to_right)

differentials_left = {}
for ind, diff in e_brane.differentials.items():
    differentials_left[ind] = matrix(
        KLRW_new,
        diff.nrows(),
        diff.ncols(),
        sparse=True,
    )
    for (i, j), entry in diff.dict(copy=False).items():
        for braid, coeff in entry:
            differentials_left[ind][i, j] += move_dots_to_right(
                coeff
            ) * KLRW_new.KLRWmonomial(
                state=KLRW_new.state(new_sequence + braid.right_state().as_tuple()),
                word=tuple(x + len(new_sequence) for x in braid.word()),
            )


left = KLRWPerfectComplex(
    KLRW_new, differentials=differentials_left, projectives=projectives_left, check=True
)

left

Ring morphism:
  From: Multivariate Laurent Polynomial Ring in r_1, r_2, u_1, t_1_2, t_2_1, u_2, x_1_1, x_2_1, x_2_2 over Rational Field
  To:   Multivariate Laurent Polynomial Ring in r_1, r_2, u_1, t_1_2, t_2_1, u_2, x_1_1, x_1_2, x_2_1, x_2_2, x_2_3, x_2_4 over Rational Field
  Defn: r_1 |--> r_1
        r_2 |--> r_2
        u_1 |--> u_1
        t_1_2 |--> t_1_2
        t_2_1 |--> t_2_1
        u_2 |--> u_2
        x_1_1 |--> x_1_2
        x_2_1 |--> x_2_3
        x_2_2 |--> x_2_4


A complex of projectives of a KLRW algebra
with differentials given by the following matrices
d_3 = 
[                                                                                                                            E_(V2, V1, V2, V2, V1, V2, W2, W2)*s4*s5*s6*E_(V2, V1, V2, V1, V2, W2, V2, W2)                                                                                                          (r_2*t_2_1*u_2^-1)*E_(V2, V1, V2, V2, V1, V2, W2, W2)*s6*s5*s7*E_(V2, V1, V2, V2, W2, V1, W2, V2)                                                                                                                       E_(V2, V1, V2, V2, V1, V2, W2, W2)*s5*s4*s6*s5*s7*E_(V2, V1, V2, V2, W2, V2, W2, V1)                                                                                                          (r_2*t_2_1*u_2^-1)*E_(V2, V1, V2, V2, V1, V2, W2, W2)*s6*s5*s4*E_(V2, V1, V2, W2, V2, V1, V2, W2)                                                                                       

In [8]:
grading_group = KLRW.grading_group
degree_shift = sum(
    grading_group.crossing_grading(v1, v2) for v1 in new_sequence for v2 in lowest_state
)

projectives_right = {
    i: [
        KLRWProjectiveModule(
            KLRW_new.state(pr.state.as_tuple() + new_sequence),
            pr.equivariant_degree + degree_shift,
        )
        for pr in projs
    ]
    for i, projs in e_brane.projectives.items()
}

variables_images = [None] * KLRW.base().ngens()
for index, var in KLRW.base().variables.items():
    if var.position is not None:
        variables_images[var.position] = KLRW_new.base().variables[index].monomial

move_dots_to_left = KLRW.base().hom(variables_images, codomain=KLRW_new.base())
print(move_dots_to_left)

differentials_right = {}
for ind, diff in e_brane.differentials.items():
    differentials_right[ind] = matrix(
        KLRW_new,
        diff.nrows(),
        diff.ncols(),
        sparse=True,
    )
    for (i, j), entry in diff.dict(copy=False).items():
        for braid, coeff in entry:
            differentials_right[ind][i, j] += move_dots_to_left(
                coeff
            ) * KLRW_new.KLRWmonomial(
                state=KLRW_new.state(braid.right_state().as_tuple() + new_sequence),
                word=braid.word(),
            )


right = KLRWPerfectComplex(
    KLRW_new,
    differentials=differentials_right,
    projectives=projectives_right,
    check=True,
)

right

Ring morphism:
  From: Multivariate Laurent Polynomial Ring in r_1, r_2, u_1, t_1_2, t_2_1, u_2, x_1_1, x_2_1, x_2_2 over Rational Field
  To:   Multivariate Laurent Polynomial Ring in r_1, r_2, u_1, t_1_2, t_2_1, u_2, x_1_1, x_1_2, x_2_1, x_2_2, x_2_3, x_2_4 over Rational Field
  Defn: r_1 |--> r_1
        r_2 |--> r_2
        u_1 |--> u_1
        t_1_2 |--> t_1_2
        t_2_1 |--> t_2_1
        u_2 |--> u_2
        x_1_1 |--> x_1_1
        x_2_1 |--> x_2_1
        x_2_2 |--> x_2_2


A complex of projectives of a KLRW algebra
with differentials given by the following matrices
d_3 = 
[                                                                                                                            E_(V2, V1, V2, W2, W2, V2, V1, V2)*s1*s2*s3*E_(V1, V2, W2, V2, W2, V2, V1, V2)                                                                                                          (r_2*t_2_1*u_2^-1)*E_(V2, V1, V2, W2, W2, V2, V1, V2)*s3*s2*s4*E_(V2, W2, V1, W2, V2, V2, V1, V2)                                                                                                                       E_(V2, V1, V2, W2, W2, V2, V1, V2)*s2*s1*s3*s2*s4*E_(V2, W2, V2, W2, V1, V2, V1, V2)                                                                                                          (r_2*t_2_1*u_2^-1)*E_(V2, V1, V2, W2, W2, V2, V1, V2)*s3*s2*s1*E_(W2, V2, V1, V2, W2, V2, V1, V2)                                                                                       

In [9]:
from klrw.perfect_complex import KLRWHomOfPerfectComplexes

rhom = KLRWHomOfPerfectComplexes(left, right)
ext = rhom.ext(0)
assert len(ext.basis()) == 1
v = ext.basis()[0]

v

3 -> 3:
[                                                                                                                                                                         E_(V2, V1, V2, V2, V1, V2, W2, W2)*s3*s2*s1*s4*s3*s2*s5*s4*s3*s6*s5*s4*s7*s6*s5*E_(V2, V1, V2, W2, W2, V2, V1, V2)                                                                                                                                                                                                                                                                                           0                                                                                                                                                                                                                                                                                           0                                                                                                                                            

In [10]:
rhom = KLRWHomOfPerfectComplexes(right,left)
ext = rhom.ext(0)
assert len(ext.basis()) == 1
u = ext.basis()[0]

u

3 -> 3:
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              (r_1^-2*r_2^-8*t_1_2^-2*t_2_1^-2*u_2^-4)*E_(V2, V1, V2, W2, W2, V2, V1, V2)*s5*s4*s3*s2*s1*s6*s5*s4*s3*s2*s7*s6*s5*s4*s3*E_(V2, V1, V2, V2, V1, V

In [11]:
rhom_left = KLRWHomOfPerfectComplexes(left, left)
ext_left = rhom_left.ext(0)

ext_left.if_homotopic_to_scalar(u*v)

True

In [12]:
rhom_right = KLRWHomOfPerfectComplexes(right, right)
ext_right = rhom_right.ext(0)

ext_right.if_homotopic_to_scalar(v*u)

True